In [3]:
import logging
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
from sqlalchemy import exc, create_engine
import pymysql
import pandas as pd

logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [2]:
try:
    conn = pymysql.connect(host=os.environ.get('IP'), user=os.environ.get('RDS_USER'), passwd=os.environ.get('RDS_PW'),
     db=os.environ.get('jacob_db'), connect_timeout=5)
except pymysql.MySQLError as e:
    logger.error("ERROR: Unexpected error: Could not connect to MySQL instance.")
    logger.error(e)
    sys.exit()


In [24]:
def get_player_stats():
    year = 2021
    url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    headers = headers[1:]

    rows = soup.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]

    stats = DataFrame(player_stats, columns = headers)
    stats['PTS'] = to_numeric(stats['PTS'])
    logger.info(f'General Stats Function Successful, retrieving {len(stats)} updated rows')
    return(stats)

In [25]:
stats = get_player_stats()

INFO:root:General Stats Function Successful, retrieving 731 updated rows


In [6]:
connection = create_engine('mysql+mysqlconnector://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)

ModuleNotFoundError: No module named 'mysql'

In [15]:
stats.to_sql(con = connection, name = "player_stats", index = False, if_exists = "replace")

In [10]:
all_tables1 = pd.read_sql_query('SHOW TABLES FROM jacob_db;', connection)

In [17]:
all_tables1

,Tables_in_jacob_db
0,aws_odds_df
1,my_python_injuries
2,my_python_table
3,player_stats


In [1]:
def sql_connection():
    try:
        connection = create_engine('mysql+pymysql://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)
        logging.info('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        logging.error('SQL Connection Failed, Error:', e)
        return e


In [11]:
engine = sql_connection()

INFO:root:SQL Connection Successful


In [12]:
all_tables1 = pd.read_sql_query('SHOW TABLES FROM jacob_db;', engine)

In [13]:
all_tables1

,Tables_in_jacob_db
0,aws_odds_df
1,my_python_injuries
2,my_python_table
3,player_stats
4,player_stats2
5,player_stats3
6,player_stats4
7,player_stats44
8,player_stats69
9,player_stats71


In [14]:
df1 = pd.read_sql_query('SELECT * FROM player_stats72;', engine)

In [15]:
df1

,Player,Team,Update,Description
0,Onyeka Okongwu,Atlanta Hawks,"Wed, Jul 21, 2021",Out (Shoulder) - The Hawks announced that Okon...
1,Jaylen Brown,Boston Celtics,"Thu, May 13, 2021",Out (Wrist) - The Celtics announced that Brown...
2,Coby White,Chicago Bulls,"Thu, Jun 10, 2021",Out (Shoulder) - The Bulls announced that Whit...
3,Jamal Murray,Denver Nuggets,"Thu, Jul 22, 2021",Out (Knee) - Murray is recovering from a torn ...
4,Klay Thompson,Golden State Warriors,"Thu, Jul 22, 2021",Out (Right Achilles) - Thompson is on track to...
5,James Wiseman,Golden State Warriors,"Thu, Jul 22, 2021",Out (Knee) - Wiseman is on track to be ready b...
6,T.J. Warren,Indiana Pacers,"Thu, Mar 25, 2021",Out (Foot) - Warren underwent foot surgery and...
7,Donte DiVincenzo,Milwaukee Bucks,"Fri, May 28, 2021",Out (Foot) - DiVincenzo suffered a tendon inju...
8,Jarrett Culver,Minnesota Timberwolves,"Thu, Apr 29, 2021",Out (Ankle) - The Timberwolves announced Culve...
9,Taurean Prince,Minnesota Timberwolves,"Thu, Apr 22, 2021",Out (Ankle) - The Cavaliers announced F Taurea...


In [28]:
stats.to_sql(con = engine, name = "player_stats44", index = False, if_exists = "replace")

In [ ]:
import os
import pandas as pd
from sqlalchemy import exc, create_engine
import pymysql

print('LOADED FUNCTIONS')


d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

def sql_connection():
    try:
        connection = create_engine('mysql+pymysql://' + os.environ.get('RDS_USER') + ':' + os.environ.get('RDS_PW') + '@' + os.environ.get('IP') + ':' + os.environ.get('PORT') + '/' + os.environ.get('RDS_DB'),
                     echo = False)
        print('SQL Connection Successful')
        return(connection)
    except exc.SQLAlchemyError as e:
        print('SQL Connection Failed, Error:', e)
        return e

print('STARTING HANDLER')
def lambda_handler(event, context):
    print(len(df))
    conn3 = sql_connection()
    df.to_sql(con=conn3, name="player_stats4", index=False, if_exists="replace")
    print('WOOT FINISHED')